## Imports and declerations

In [3]:
import pickle
import os
import string

import re
import math
import pprint
from IPython.core.interactiveshell import InteractiveShell
import numpy as np
import copy
from copy import deepcopy
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
import nltk
InteractiveShell.ast_node_interactivity = "all"

## Data Structure:

In [15]:

class VectorSpaceModel(object):
    
    def tf_natural(self, tf, doc):
    #return tf
        return tf / sum(doc.values())
    def idf_no(self, term):
        return 1
    def idf_idf(self, term):
        return math.log10((len(self.docs.keys()) / (len(self.index[term]))) + 1 )
    def norm_cosine(doc):
        return 1 / (sum([tf**2 for tf in doc.values()]) + 1)
    def norm_no(self, doc):
        return 1  
    def __init__(self, tf_func = 'natural', idf_func = 'idf', norm_func='none'):
        self.tf_functions = {
            'natural': self.tf_natural,
            # 'logarithm': lambda tf, doc : (0 if tf==0 else 1 + math.log10(tf)),
            # 'augmented': lambda tf, doc : (0.5 + ((0.5 * tf)/(self.find_max_tf(doc)))),
            # 'boolean' : lambda tf, doc : (1 if tf > 0 else 0),
            # 'log_ave': self.tf_log_ave

        }
        self.idf_functions = {
            'no': self.idf_no,
            'idf':self.idf_idf,
            'prob_idf' : self.prod_idf
        }
        self.normailization_functions = {
            'none' : self.norm_no,
            'cosine' : self.norm_cosine,
#             'pivoted_unique' : lambda 
        }
        self.tf_func = self.tf_functions[tf_func]
        self.idf_func = self.idf_functions[idf_func]
        self.norm_func = self.normailization_functions[norm_func]

        self.vocab = {}
        self.vocab_idf = {}
        self.docs = {}
        self.docs_char_length = {}
        self.occurrance = {}
        self.occurrance2 = {}
        self.cdocs = {}
        self.index = {}
        def tf_log_ave(self, tf, doc):
    #         print(tf)
            return ( (1+math.log10((tf*sum(doc.values()))+1)) / (1 + math.log10(self.find_avg_tf(doc))))
    def prod_idf(self, term):
#         print('prob idf')
#         print(math.log10((len(self.docs.keys()) - (len(self.index[term])) + 1)/(len(self.index[term]))))
        if 0 > math.log10((len(self.docs.keys()) - (len(self.index[term])) + 1)/(len(self.index[term]))):
            return 0;
        else:
            return math.log10((len(self.docs.keys()) - (len(self.index[term])) + 1)/(len(self.index[term])))    
    
    
    def find_avg_tf(self, doc):
        csum = 0
        cnt = 0
#         print(f'Doc sum : {sum(doc.values()) }')
#         print(sum(doc.values()) / len(doc.keys()))
        return (sum(doc.values()) / len(doc.keys()))
    
    def find_max_tf(self, doc):
        max_tf = 0
        max_term = None
        for term, tf in doc.items():
            if tf > max_tf:
                max_tf = tf
                max_term = term
        return max_tf

    
    
    def create_doc(self, docId):
        self.docs[docId] = dict.fromkeys(self.vocab, 0)
        
    def add_term(self, term, docId, position):
        if term not in self.vocab.keys():
            self.vocab[term] = 1
            for Id, docList in self.docs.items():
                self.docs[Id][term] = 0
        else:
            self.vocab[term]+=1
        
        if term in self.docs[docId].keys():
            self.docs[docId][term] += 1
        else:
            self.docs[docId][term] = 1
            
        if docId not in self.occurrance.keys():
            self.occurrance[docId] = {}
            self.occurrance[docId][term] = []
            self.occurrance[docId][term].append(position)
        else:
            if term not in self.occurrance[docId].keys():
                self.occurrance[docId][term] = []
                self.occurrance[docId][term].append(position)
            else:
                self.occurrance[docId][term].append(position)
                
        if term in self.index.keys():
            self.index[term].add(docId)
        else:
            self.index[term] = set()
            self.index[term].add(docId)
#         if docId not in self.occurrance.keys():
#             self.occurrance[docId] = []
#         self.occurrance[docId].append(position)
    
    def get_query_vector(self, query_terms):
        query_vector_hash = dict.fromkeys(self.vocab, 0)
#         print(query_vector_hash)
        query_terms = [lem.lemmatize(word.lower()) for word in query_terms]
        print(query_terms)
        for term in query_terms:
            if term in query_vector_hash.keys():
                query_vector_hash[term] += 1

        
        words_in_query = len(query_terms)
        tf = dict.fromkeys(self.vocab_idf, 0) 
        tf_idf = dict.fromkeys(self.vocab_idf, 0)
        
        for term,term_cnt in query_vector_hash.items():
            if term_cnt <= 0:
                continue
#                 print(term)
#                 print(term_cnt)
#                 tf[term] = term_cnt / words_in_query
#                 idf[term] = len(self.docs.keys()) / (self.vocab[term])

            tf[term] = self.tf_func( term_cnt,query_vector_hash )
            tf_idf[term] = tf[term] *  self.vocab_idf[term]
            print(f'tf_idf {term}: {tf_idf[term]}')
        new_query_vector_hash = {
            'tf': tf,
            'idf':self.vocab_idf,
            'tf_idf':tf_idf
        }
    
    
        return new_query_vector_hash
    
    def dot_product(self, v_hash_1, v_hash_2):
#         print(len(v_hash_1))
#         print(len(v_hash_2))
#         print('vhash')
#         print(v_hash_2)
        return sum([v_hash_2[x]*y for x,y in v_hash_1.items()])
        
    def get_magnitude(self, v_hash):
        
        mag = sum([x**2 for x in v_hash.values()]) ** 0.5 
        if mag == 0:
            return 1
        return mag
    
    
    def get_cosine_sim(self, v_hash_x, v_hash_y):
        return (self.dot_product(v_hash_x,v_hash_y)) / (self.get_magnitude(v_hash_x) * self.get_magnitude(v_hash_y)) 
            
    def get_ranking(self, query_vector_hash):
        ranked_docs = []
        for docId,doc_vector_hash in self.cdocs.items():
            cosine_sim = self.get_cosine_sim(doc_vector_hash['tf_idf'], query_vector_hash['tf_idf'])
            ranked_docs.append((docId, cosine_sim))
        ranked_docs = sorted(ranked_docs, key=lambda x:x[1])
        return ranked_docs
    
    def calculate_tf_idf(self):
        cdocs = {}
        self.vocab_idf = dict.fromkeys(self.vocab, 0)
        for term, term_cnt in self.vocab.items():
            self.vocab_idf[term] = self.idf_func(term)
            
        for docId,doc in self.docs.items():
#             print(doc.values())
            words_in_d = sum(doc.values())
            tf = {}
        
            tf_idf = {}
            for term,term_cnt in doc.items():
#                 tf[term] = term_cnt / words_in_d
#                 idf[term] = len(self.docs.keys()) / (self.vocab[term])
                tf[term] =self.tf_func(term_cnt, doc)
                tf_idf[term] = tf[term] * self.vocab_idf[term]
            cdocs[docId] = {
                'tf': tf,
                'tf_idf':tf_idf,
                'total_words' : words_in_d
            }
        self.cdocs = cdocs
        return self.cdocs
            
        

In [16]:
test_space = VectorSpaceModel()

d1= "Music is a universal language"
d2= "Music is a miracle"
d3= "Music is a universal feature of the human experience"
# test_space.create_doc(1)
# for word in d1.split(' '):
#     test_space.add_term(word, 1)
# test_space.create_doc(2)
# for word in d2.split(' '):
#     test_space.add_term(word, 2)
# test_space.create_doc(3)
# for word in d3.split(' '):
#     test_space.add_term(word, 3)
# test_space.calculate_tf_idf()


## Helper Functions:

In [17]:
# Remove Punctuation
def remove_punctuation(word):
    return word.translate(word.maketrans('','',string.punctuation))

# Clean Query Term
def clean_word(word):
    # Case Folding
    word = word.lower()
     # Filter non-ASCII characters
    word = ''.join(filter(lambda x: x in printable, word))
#     print(word)
    # Remove Punctuations
    if word != '(' and word != ')':
        word = remove_punctuation(word)
#     print(word)
    if re.match('\d+[A-Za-z]+',word):
        word = re.split('\d+',word)[1]
    if re.match('[A-Za-z]+\d+',word):
        word = re.split('\d+',word)[0]
#     print(word)
    word = ps.stem(word)
#     print(word)
    return word

## Term - Document Indexing:

In [18]:
vocab = set()
doc_contents = []

vector_space = VectorSpaceModel(tf_func='natural',idf_func='idf')
printable = set(string.printable) 
raw_data = []
# Printable characters are
# 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
# !"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c


lem = WordNetLemmatizer() 

stop_words = set()
with open('Stopword-List.txt', 'r') as stop_word_file:
    lines = stop_word_file.readlines()
    for line in lines:
        stop_words.add(line.split('\n')[0])
    stop_words.remove('')

    
print('Loading : ', end='')
for file_number in range(0, 56):
    vector_space.create_doc(file_number)
    with open(f'data/Trump Speechs/speech_{file_number}.txt', 'r') as file1:
        lines = file1.readlines()
#         print(f'File Number : speech_{file_number}.txt' )
#         print(lines[0])
        position = {'doc':file_number,'row':0, 'col':0, 'token_no':0}
        
#         {
#             'total_count' : 0,
#             'postings' : {
#                 'count':0,
#                 'doc_id':0,
#                 'positions':[]
#             }
#         }
        
        for line_no,line in enumerate(lines):
            # Skip Heading Line
            if line_no == 0:
                continue
            doc_set = set()
            # split words at . , whitespace ? ! : ;
            position['row'] = line_no 
            position['col'] = 0
            symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
            for i in symbols:
                line = line.replace(i, ' ')
            raw_data.append(line)
            for word in re.split('[.\s,?!:;-]', line):
                
                position['col'] += len(word) + 1
                position['token_no'] += 1
                # Case Folding
                word = word.lower()
                
                # Filter non-ASCII characters
                word = ''.join(filter(lambda x: x in printable, word))
                
                # Remove Punctuations
                word = remove_punctuation(word)
                
                if re.match('\d+[A-Za-z]+',word):
                    word = re.split('\d+',word)[1]
                if re.match('[A-Za-z]+\d+',word):
                    word = re.split('\d+',word)[0]
                
                if len(word) == 0 or len(word) == 1 or word == '' or word == ' ':
                    continue
                if word in stop_words:
                    continue
                
                word = lem.lemmatize(word)
                    
                vocab.add(word)
                
                doc_set.add(word)
                
                vector_space.add_term(word, file_number, deepcopy(position))
                
        doc_contents.append(doc_set)
        print('*', end='')
doc_term_tf_idf = vector_space.calculate_tf_idf()
print('Done')

Loading : ********************************************************Done


In [19]:
print('Total Vocabulary Size ')
print(len(vector_space.index.keys()))
print('Total Number of Documents ')
print(len(vector_space.docs))

Total Vocabulary Size 
6249
Total Number of Documents 
56


In [20]:
# To save Inverted Index in File
with open('pickled/vector_space.p', 'wb') as index_file:
    pickle.dump(vector_space, index_file)

In [21]:
query = "Hillary clinton"
query_terms = [lem.lemmatize(x) for x in query.split(' ')]
print(query_terms)

query_vector = vector_space.get_query_vector(query_terms)

# cs = vector_space.get_cosine_sim(vector_space.docs[1], query_vector)
# print([(x,y) for x,y in query_vector.items() if y > 0])
# ranked = vector_space.get_ranking(vector_space.get_query_vector(query_terms))
# ranked.reverse()
# ranked = set([x for x,y in ranked if y > 0.0005])
# print(ranked)

vector_space.index['global']

['Hillary', 'clinton']
['hillary', 'clinton']
tf_idf clinton: 0.15657531416991727
tf_idf hillary: 0.15871020592607524


{1, 2, 9, 16, 20, 31, 32, 36, 40, 42, 43, 44, 46, 52}

# Query Tests:

In [22]:
test = [(None, None) for x in range(0, 17)] 
test[0] =("massive inflow of refugees",
{'32', '50', '49', '47', '46', '29', '48', '54', '41', '40', '30', '39', '12', '52', '37', '44', '31', '38', '20'}
         )
test[1] =('pakistan afghanistan',
{'3', '22', '16', '17', '4', '1'}
         )
test[2] =('Hillary Clinton',
{'4', '12', '36', '42', '33', '20', '29', '35', '16', '22', '47', '34', '14', '21', '45', '46', '41', '11', '3', '51', '53', '40', '37', '17', '48', '39', '5', '28', '10', '18', '54', '44', '25', '43', '49', '24', '30', '32', '8', '50', '9', '26', '7', '19', '52', '31', '27', '6', '2', '38', '1', '55'}
         )
test[3] =('personnel policies',

{'5', '18', '11', '25', '10', '22', '29', '27', '17'}
         )

test[4] =('united plane',
{'38', '1', '25', '41', '48', '29', '3', '17', '49', '20', '4', '52', '13', '39', '46', '22', '40', '36', '28', '12', '45', '26', '2', '51', '50', '19', '24', '5', '47', '31', '35', '21', '37', '55', '9', '33', '44', '54', '34', '7', '32', '43', '30', '16', '27', '18', '11', '10', '8'}
         )

test[5] =('develop solutions',
{'38', '23', '2', '17', '32', '18', '51', '16', '39', '35', '21', '27', '33', '20', '24', '5', '3', '9', '40', '52', '30'}
         )

test[6] =('developments praised',
{'36', '44', '8'}
         )
test[7] =("muslims",
{'4', '3'}
         )
test[8] =('American Energy Revolution',

{'13', '28', '35', '33', '51', '27', '12', '36', '34', '14', '21', '29', '31', '30', '25', '32', '26', '24', '11', '54', '49', '16', '46', '48', '10', '18', '4', '50', '42', '5', '23', '47', '20', '43', '37', '39', '41', '45', '2', '22', '3', '40', '7', '17', '52', '44', '53', '19', '9', '55', '1', '15', '8'}
         )
test[9] =('Future of new America',
{'51', '14', '35', '26', '13', '50', '33', '4', '12', '31', '20', '32', '29', '34', '41', '27', '46', '30', '40', '25', '17', '42', '16', '24', '54', '2', '43', '21', '7', '18', '52', '49', '5', '47', '45', '10', '36', '38', '44', '11', '3', '39', '22', '9', '48', '37', '23', '15', '1', '19', '55', '6', '8'}
         )
test[10] =('Hillary clinton is the worst looser',
{'4', '12', '36', '42', '33', '20', '29', '35', '16', '22', '47', '34', '14', '21', '45', '46', '41', '11', '3', '51', '53', '40', '37', '17', '48', '39', '5', '28', '10', '18', '54', '44', '25', '43', '49', '24', '30', '32', '8', '50', '9', '26', '7', '19', '52', '31', '27', '6', '2', '38', '1', '55'}
          )
test[11] =('no patience for injustice',
{'11', '22', '16', '7', '15'}
          )
test[12] =('Global interests',
{'10', '16', '27', '42', '21', '11', '25', '22', '47', '41', '48', '54', '18', '40', '24', '39', '8'}
          )
test[13] =('pakistan afghanistan aid',
{'3', '22', '42', '29', '41', '16', '40', '39', '17', '4', '1'}
          )
test[14] =('biggest plane wanted hour',
{'52', '1', '19'}
          )
test[15] =('near architect box',
{'54', '23', '18', '24', '39', '43', '51', '4', '9', '17', '11'}
          )
test[16] =('peaceful change',
{'14', '10', '4', '30', '3', '11', '20', '16', '50', '52', '48', '17', '7', '2', '22', '29', '13', '53', '41', '46', '32', '21', '39', '25', '33', '54', '42', '5', '36', '31', '43', '51', '49', '23', '47', '35', '37', '12', '45', '8'}
         )
cumulative_precision_normal = 0
cumulative_recall_normal = 0 
cumulative_precision_sciket = 0
cumulative_recall_sciket = 0 
cumulative_precision_group = 0
cumulative_recall_group = 0 

cnt = 0
for t in test:
    cnt+=1
    normal_cnt = [0, 0 , 0]
    # my custom model
    query = t[0]
    query_terms = [lem.lemmatize(x.lower()) for x in query.split(' ')]
    print('\t' + str(query_terms))
    # cs = vector_space.get_cosine_sim(vector_space.docs[1], query_vector)
    # print([(x,y) for x,y in query_vector.items() if y > 0])
    ranked = vector_space.get_ranking(vector_space.get_query_vector(query_terms))
    ranked.reverse()
    print(str([(x,y) for x,y in ranked if y > 0.0000]))
    ranked = set([str(x) for x,y in ranked if y > 0.0000])
    
    print('Normal results')
    print(f'results length :  {len(ranked)}')
    print('Result Differnece : ')
    print(t[1].difference(ranked))
    print('Symmetric Difference : ')
    print(t[1].symmetric_difference(ranked))
    print()
    normal_cnt = [len(t[1]), len(ranked.difference(t[1])), len(t[1].difference(ranked))]
    normal_p = normal_cnt[0] / (normal_cnt[0] + normal_cnt[1])
    normal_r = normal_cnt[0] / (normal_cnt[0] + normal_cnt[2])
    print(f'precision : {normal_p}')
    print(f'recall : {normal_r}')
    cumulative_precision_normal += normal_p
    cumulative_recall_normal += normal_r
    


    
f1_normal = (2 * (cumulative_precision_normal / cnt) * (cumulative_recall_normal / cnt) ) / (cumulative_recall_normal+cumulative_precision_normal)


print(f'normal f1 : {f1_normal}')


	['massive', 'inflow', 'of', 'refugee']
['massive', 'inflow', 'of', 'refugee']
tf_idf of: 0.0752574989159953
tf_idf massive: 0.09666474956558715
tf_idf refugee: 0.11203938969608647
tf_idf inflow: 0.18833191666465288
[(41, 0.14238395203047455), (54, 0.13614270772046327), (48, 0.13577266734153465), (29, 0.13318415662080998), (47, 0.13296037933069274), (40, 0.13285554788949944), (7, 0.12825833096070022), (49, 0.12809037874567442), (39, 0.12617821737443477), (32, 0.12610667649337276), (24, 0.12493493864380238), (52, 0.12444365156394711), (20, 0.12361639684231786), (46, 0.12338069327849539), (12, 0.12282088533663452), (14, 0.11852880246109719), (4, 0.11848620892779728), (16, 0.1183355922760708), (18, 0.11779896665405311), (25, 0.11775128353511148), (37, 0.11774530330133837), (9, 0.11722407323682882), (21, 0.11522891772785722), (3, 0.11379539932382887), (30, 0.11362809063388735), (17, 0.10805046214024351), (50, 0.10510484882625705), (11, 0.10474089871210134), (22, 0.10385616011878274), (27, 

[(31, 0.035939866291364315), (36, 0.017815469920515203), (44, 0.014438744337525554), (8, 0.003863405462203211)]
Normal results
results length :  4
Result Differnece : 
set()
Symmetric Difference : 
{'31'}

precision : 0.75
recall : 1.0
	['muslim']
['muslim']
tf_idf muslim: 0.9542425094393249
[(3, 0.06847745290157702), (4, 0.040067128561783606), (9, 0.03266160626131962), (2, 0.022462818788261157), (20, 0.01559631871326888), (6, 0.008682179111972119), (7, 0.008577975818554469)]
Normal results
results length :  7
Result Differnece : 
set()
Symmetric Difference : 
{'2', '20', '9', '7', '6'}

precision : 0.2857142857142857
recall : 1.0
	['american', 'energy', 'revolution']
['american', 'energy', 'revolution']
tf_idf american: 0.10165342976413785
tf_idf energy: 0.12466262584883647
tf_idf revolution: 0.3180808364797749
[(31, 0.14087897587989007), (30, 0.08667228563724078), (13, 0.07443312059301091), (33, 0.07350047489759047), (34, 0.07311723865743368), (27, 0.07069400291502571), (35, 0.068003

[(0, 0.027337808441140084), (8, 0.026750921228480887), (52, 0.021981219420626547), (35, 0.021915179594799127), (19, 0.017810235973704694), (51, 0.014185321005353902), (24, 0.01400687911406263), (34, 0.013758535352917368), (43, 0.013721051728529965), (41, 0.013718731214653206), (11, 0.011615738674330958), (40, 0.011513373480500303), (30, 0.010988317196687901), (2, 0.010891490037294584), (9, 0.01057930201176322), (5, 0.010270483707070744), (28, 0.010110688478088246), (1, 0.008523843710632039), (39, 0.008449755468759684), (36, 0.0082022821961308), (46, 0.008183984281999155), (27, 0.007691012404753345), (25, 0.007500510979558549), (29, 0.007152883477022979), (33, 0.007048966866187816), (53, 0.006641393000257859), (15, 0.0065912060769672), (3, 0.006448067638902592), (26, 0.006327306649801213), (54, 0.006220497698612816), (32, 0.006125552916560202), (6, 0.0055344023922151795), (55, 0.005414356635241327), (17, 0.005315193273571361), (10, 0.005309195239509735), (42, 0.005294333290353419), (18,

In [24]:
query = input('Enter Your Query  : ')
lem = WordNetLemmatizer()
query_terms = [lem.lemmatize(x.lower()) for x in query.split(' ')]
print('\t' + str(query_terms))
# cs = vector_space.get_cosine_sim(vector_space.docs[1], query_vector)
# print([(x,y) for x,y in query_vector.items() if y > 0])
ranked = vector_space.get_ranking(vector_space.get_query_vector(query_terms))
ranked.reverse()
print(str([(x,y) for x,y in ranked if y > 0.0005]))
ranked = set([str(x) for x,y in ranked if y > 0.0005])

Enter Your Query  : muslims
	['muslim']
['muslim']
tf_idf muslim: 0.9542425094393249
[(3, 0.06847745290157702), (4, 0.040067128561783606), (9, 0.03266160626131962), (2, 0.022462818788261157), (20, 0.01559631871326888), (6, 0.008682179111972119), (7, 0.008577975818554469)]


# References :

http://www.pyregex.com/
http://cs231n.github.io/python-numpy-tutorial/

https://www.online-utility.org/text/analyzer.jsp

https://stackoverflow.com/questions/2118261/parse-boolean-arithmetic-including-parentheses-with-regex

https://regex101.com/r/M8z3U4/1

https://iq.opengenus.org/porter-stemmer/

https://unnikked.ga/how-to-build-a-boolean-expression-evaluator-518e9e068a65

https://ruslanspivak.com/lsbasi-part7/

https://github.com/gintas/django-picklefield